In [3]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
client = openai.Client()

In [4]:
assistant = client.beta.assistants.create(
    name='Tutor de Matemática',
    instructions='Você é um tutor pessoal de matemática. Escreva e execute códigos para responder as perguntas de matemática que lhe forem passadas.',
    tools=[{'type':'code_interpreter'}],
    model='gpt-3.5-turbo-0125'
)

In [5]:
thread = client.beta.threads.create()

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='Se eu jogar um dado honesto 1000 vezes, qual a probabilidade de eu obter exatamente 150 vezes o número 6? Resolva com um código.'
)

In [8]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Adriano Soares e ele é um usuário Premium.'
)

In [9]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [10]:
run.status

'completed'

In [12]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens.data[0].content[0].text.value)
else:
    print('Erro', run.status)

A probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes é aproximadamente 0.0126, ou seja, aproximadamente 1.26%.


# Analisando os passos do modelo

In [13]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [20]:
run_steps.data[0]

RunStep(id='step_7k2rlzma17sgkBWEXpWJQawW', assistant_id='asst_FqkLlh8olRJUUy5b8JHqAsDA', cancelled_at=None, completed_at=1741886793, created_at=1741886793, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_3LWLNi20whlyWsyfJUBLFC81', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_RbpyC71B2PnccjgIZcxZ1wlV'), type='message_creation'), thread_id='thread_zJoWud86IpuV9GfJePa8ur9z', type='message_creation', usage=Usage(completion_tokens=48, prompt_tokens=464, total_tokens=512, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), expires_at=None)

In [15]:
for step in run_steps.data:
    print('===Step: ', step)

=== Step: message_creation
=== Step: tool_calls
=== Step: message_creation


In [23]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('-----')
            print(tool_call.code_interpreter.input)
            print('-----')
            print('Result')
            # Certifique-se de que outputs não esteja vazio antes de acessar
            if tool_call.code_interpreter.outputs:
                print(tool_call.code_interpreter.outputs[0].logs)
            else:
                print("No outputs available.")
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)


=== Step: message_creation
Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes, podemos usar a distribuição de probabilidade binomial. A fórmula da distribuição binomial é dada por:

\[ P(X = k) = \binom{n}{k} \times p^k \times (1-p)^{n-k} \]

onde:
- \( P(X = k) \) é a probabilidade de obter exatamente k sucessos em n tentativas,
- \( n = 1000 \) é o número de tentativas,
- \( k = 150 \) é o número de sucessos desejados (obter o número 6),
- \( p = \frac{1}{6} \) é a probabilidade de sucesso em uma única tentativa (sair o número 6),
- \( \binom{n}{k} \) é o número de combinações de n elementos tomados k a k.

Vou calcular a probabilidade para você.

=== Step: tool_calls
-----
from math import comb

n = 1000
k = 150
p = 1/6

# Calculando a probabilidade
probabilidade = comb(n, k) * (p ** k) * ((1 - p) ** (n - k))
probabilidade
-----
Result
No outputs available.

=== Step: message_creation
A probabilidade de obter exatamente 150 ve